# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-08-2022'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-08-2022'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-01-09 04:21:46,33.93911,67.709953,158309,7369,NaN,NaN,Afghanistan,406.667728,4.654821
1,NaN,NaN,NaN,Albania,2022-01-09 04:21:46,41.15330,20.168300,214905,3233,NaN,NaN,Albania,7467.683647,1.504386
2,NaN,NaN,NaN,Algeria,2022-01-09 04:21:46,28.03390,1.659600,221742,6330,NaN,NaN,Algeria,505.670983,2.854669
3,NaN,NaN,NaN,Andorra,2022-01-09 04:21:46,42.50630,1.521800,26408,141,NaN,NaN,Andorra,34178.476671,0.533929
4,NaN,NaN,NaN,Angola,2022-01-09 04:21:46,-11.20270,17.873900,89251,1819,NaN,NaN,Angola,271.558061,2.038072


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22,1/7/22,1/8/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,158056,158084,158107,158189,158183,158205,158245,158275,158300,158309
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,208899,210224,210224,210885,210885,212021,212021,213257,214905,214905
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,218037,218432,218818,219159,219532,219953,220415,220825,221316,221742


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22,1/7/22,1/8/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7356,7356,7356,7361,7364,7365,7367,7367,7368,7369
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3212,3217,3217,3220,3220,3224,3224,3228,3233,3233
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6271,6276,6284,6291,6298,6303,6310,6318,6325,6330


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22,1/7/22,1/8/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22,1/7/22,1/8/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,10953,11018,11018,11184,11256,11347,11478,11638,11789,11856
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,39583,39911,39911,40396,40549,40838,41312,41855,42391,42738
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,3819,3860,3860,3932,3961,3999,4036,4101,4150,4180


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22,1/7/22,1/8/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,160,160,160,160,160,161,161,161,162,162
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,593,593,593,593,593,594,594,598,601,603
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,81,81,81,81,81,81,81,83,83,83


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
673,1001.0,Autauga,Alabama,US,2022-01-09 04:21:46,32.539527,-86.644082,11856,162,NaN,NaN,"Autauga, Alabama, US",21221.070719,1.366397
710,1075.0,Lamar,Alabama,US,2022-01-09 04:21:46,33.779950,-88.096680,2674,60,NaN,NaN,"Lamar, Alabama, US",19369.793553,2.243829
711,1077.0,Lauderdale,Alabama,US,2022-01-09 04:21:46,34.901719,-87.656247,17002,320,NaN,NaN,"Lauderdale, Alabama, US",18335.148659,1.882132


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,59767221,837262,0.0
India,35368372,483463,0.0
Brazil,22328252,619654,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,59767221,837262,0.0,58929959.0,2022-01-09 04:21:46,37.934269,-91.444083
India,35368372,483463,0.0,34884909.0,2022-01-09 04:21:46,23.088275,81.806127
Brazil,22328252,619654,0.0,21708598.0,2022-01-09 04:21:46,-12.669522,-48.480493
United Kingdom,14422067,150537,0.0,14271530.0,2022-01-09 04:21:46,28.164647,-33.966498
France,11921925,126337,0.0,11795588.0,2022-01-09 04:21:46,6.430808,-34.730285


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,6037516,77041,0.0
Texas,4999623,76461,0.0
Florida,4633077,62688,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,6037516,77041,0.0,5960475.0,2022-01-09 04:21:46,37.843962,-120.728594
Texas,4999623,76461,0.0,4923162.0,2022-01-09 04:21:46,31.660643,-98.653069
Florida,4633077,62688,0.0,4570389.0,2022-01-09 04:21:46,28.940755,-82.700744
New York,4057133,60441,0.0,3996692.0,2022-01-09 04:21:46,42.544151,-75.474183
Illinois,2382437,30794,0.0,2351643.0,2022-01-09 04:21:46,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1921890,27772,0.0
Florida,Miami-Dade,965111,6472,0.0
Arizona,Maricopa,926432,13914,0.0
Illinois,Cook,917379,12202,0.0
Texas,Harris,738421,9813,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1921890,27772,0.0,1894118.0,2022-01-09 04:21:46,34.308284,-118.228241,6037.0
Florida,Miami-Dade,965111,6472,0.0,958639.0,2022-01-09 04:21:46,25.611236,-80.551706,12086.0
Arizona,Maricopa,926432,13914,0.0,912518.0,2022-01-09 04:21:46,33.348359,-112.491815,4013.0
Illinois,Cook,917379,12202,0.0,905177.0,2022-01-09 04:21:46,41.841448,-87.816588,17031.0
Texas,Harris,738421,9813,0.0,728608.0,2022-01-09 04:21:46,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,158275,213257,220825,25289,87625,4715,6025303,345389,762458,1312939,...,14100750,430958,199507,7,445680,1843563,471414,10152,274087,220178
2022-01-07,158300,214905,221316,26408,88775,4715,6135836,345518,860550,1321725,...,14280816,437062,199758,7,446026,1859841,471414,10159,278276,221282
2022-01-08,158309,214905,221742,26408,89251,4844,6237525,345713,972457,1329130,...,14422067,443436,199972,7,446427,1876394,471414,10178,281576,221918


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,7367,3228,6318,141,1799,119,117386,7993,2321,13823,...,149993,6186,1485,1,5341,33644,4954,1986,3782,5108
2022-01-07,7368,3233,6325,141,1809,119,117428,7997,2337,13830,...,150223,6188,1488,1,5345,33877,4954,1986,3790,5136
2022-01-08,7369,3233,6330,141,1819,119,117465,7999,2366,13844,...,150537,6189,1492,1,5348,34117,4954,1986,3798,5148


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22,1/7/22,1/8/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,10953,11018,11018,11184,11256,11347,11478,11638,11789,11856
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,2603,2613,2613,2617,2624,2625,2634,2650,2661,2674
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,15768,15863,15863,16034,16084,16154,16396,16613,16849,17002
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,5046,5071,5071,5130,5145,5174,5231,5295,5354,5385
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,26789,26975,26975,27323,27438,27892,28208,28620,28971,29355


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-06,944876,161678,10,1430241,596409,5894761,979437,563635,195600,49,...,1462996,4897362,672567,71942,11833,1202727,913980,347554,1170504,119093
2022-01-07,957848,165353,10,1445129,604843,6003152,989409,573711,198507,49,...,1462996,4949933,682036,73802,12138,1221036,931071,351688,1186066,119857
2022-01-08,966180,165353,10,1461633,613014,6037516,989409,573711,202236,49,...,1462996,4999623,682036,76421,12541,1221036,931071,355872,1186066,119857


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-01-06       11638   41855    4101  4730  11587    1783   3838   24282   
2022-01-07       11789   42391    4150  4815  11690    1806   3896   24507   
2022-01-08       11856   42738    4180  4857  11749    1831   3919   24678   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-01-06         6603     3511  ...    5747   1595     6139     1547   
2022-01-07         6689     3550  ...    5770   1598     6167     1551   
2022-01-08         6749     3569  ...    5770   1598     6167     1551   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-01-06           8471  6836  4245          0     1909   1261  
2022-01-07           8506  7025  4264          0     1915   1263  
2022-01-08           8506  7025  4264          0     1915   1263  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-06,16543,979,0,24586,9278,76792,10402,9281,2286,0,...,20941,76260,3868,482,89,15642,9974,5392,11376,1572
2022-01-07,16585,980,0,24616,9298,77026,10425,9377,2330,0,...,20941,76365,3887,482,89,15651,10004,5421,11402,1572
2022-01-08,16624,980,0,24704,9314,77041,10425,9377,2365,0,...,20941,76461,3887,483,89,15651,10004,5421,11402,1572


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-01-06         161     598      83   95    198      46    102     532   
2022-01-07         162     601      83   95    199      46    102     533   
2022-01-08         162     603      83   95    202      46    102     535   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-01-06          147       66  ...     121     37       63       24   
2022-01-07          147       66  ...     121     37       63       24   
2022-01-08          147       66  ...     121     37       63       24   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-01-06            110    14    34          0       39     16  
2022-01-07            110    14    34          0       39     16  
2022-01-08            110    14    34          0       39     16  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,0.000190,0.005830,0.001860,0.000000,0.011416,0.051048,0.018528,0.000388,0.114332,0.006789,...,0.013093,0.012980,0.000732,0.0,0.000000,0.014217,0.0,0.000591,0.015171,0.005117
2022-01-07,0.000158,0.007728,0.002223,0.044248,0.013124,0.000000,0.018345,0.000373,0.128652,0.006692,...,0.012770,0.014164,0.001258,0.0,0.000776,0.008830,0.0,0.000690,0.015283,0.005014
2022-01-08,0.000057,0.000000,0.001925,0.000000,0.005362,0.027359,0.016573,0.000564,0.130041,0.005603,...,0.009891,0.014584,0.001071,0.0,0.000899,0.008900,0.0,0.001870,0.011859,0.002874


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,0.000000,0.001241,0.001268,0.0,0.005590,0.0,0.000341,0.000501,0.008254,0.001522,...,0.001549,0.000809,0.000000,0.0,0.000000,0.005049,0.0,0.000504,0.002651,0.003142
2022-01-07,0.000136,0.001549,0.001108,0.0,0.005559,0.0,0.000358,0.000500,0.006894,0.000506,...,0.001533,0.000323,0.002020,0.0,0.000749,0.006925,0.0,0.000000,0.002115,0.005482
2022-01-08,0.000136,0.000000,0.000791,0.0,0.005528,0.0,0.000315,0.000250,0.012409,0.001012,...,0.002090,0.000162,0.002688,0.0,0.000561,0.007084,0.0,0.000000,0.002111,0.002336


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-06,0.013544,0.00000,0.0,0.007523,0.013229,0.014908,0.008144,0.015903,0.013640,0.0,...,0.0,0.009352,0.013430,0.031249,0.060305,0.013346,0.016622,0.014439,0.012503,0.008425
2022-01-07,0.013729,0.02273,0.0,0.010409,0.014141,0.018388,0.010181,0.017877,0.014862,0.0,...,0.0,0.010735,0.014079,0.025854,0.025775,0.015223,0.018700,0.011895,0.013295,0.006415
2022-01-08,0.008699,0.00000,0.0,0.011420,0.013509,0.005724,0.000000,0.000000,0.018785,0.0,...,0.0,0.010039,0.000000,0.035487,0.033202,0.000000,0.000000,0.011897,0.000000,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-01-06      0.013940  0.013144  0.016105  0.010684  0.007828  0.017694   
2022-01-07      0.012975  0.012806  0.011948  0.017970  0.008889  0.012900   
2022-01-08      0.005683  0.008186  0.007229  0.008723  0.005047  0.013843   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-01-06      0.017228  0.009773  0.021030  0.006883  ...  0.002792   
2022-01-07      0.015112  0.009266  0.013024  0.011108  ...  0.004002   
2022-01-08      0.005903  0.006978  0.008970  0.005352  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-01-06      0.003145  0.004582  0.003243   0.005699  0.038748  0.005924   
2022-01-07      0.001881  0.004561  0.002586   0.004132  0.027648  0.004476   
2022-01-08      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-01-06            NaN  0.003153  0.000794  
2022-01-07            NaN  0.003143  0.001586  
2022-01-08            NaN  0.000000  0.000000  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-06,0.002424,0.000000,NaN,0.000651,0.002377,0.000417,0.004636,0.000000,0.000000,NaN,...,0.0,0.001681,0.003372,0.000000,0.0,0.000704,0.006560,0.003723,0.005036,0.0
2022-01-07,0.002539,0.001021,NaN,0.001220,0.002156,0.003047,0.002211,0.010344,0.019248,NaN,...,0.0,0.001377,0.004912,0.000000,0.0,0.000575,0.003008,0.005378,0.002286,0.0
2022-01-08,0.002352,0.000000,NaN,0.003575,0.001721,0.000195,0.000000,0.000000,0.015021,NaN,...,0.0,0.001257,0.000000,0.002075,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour Bibb    Blount Bullock Butler   
2022-01-06      0.000000  0.006734  0.024691  0.0  0.000000     0.0    0.0   
2022-01-07      0.006211  0.005017  0.000000  0.0  0.005051     0.0    0.0   
2022-01-08      0.000000  0.003328  0.000000  0.0  0.015075     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2022-01-06      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2022-01-07      0.001880      0.0      0.0  ...     0.0    0.0      0.0   
2022-01-08      0.003752      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-01-06          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-01-07          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-01-08          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,0.000194,0.003745,0.001905,0.008754,0.014215,0.031474,0.015901,0.000334,0.115505,0.006139,...,0.013817,0.011022,0.000590,9.934107e-09,0.000297,0.012154,0.000533,0.000493,0.014994,0.005829
2022-01-07,0.000176,0.005736,0.002064,0.026501,0.013669,0.015737,0.017123,0.000354,0.122079,0.006415,...,0.013293,0.012593,0.000924,4.967054e-09,0.000537,0.010492,0.000267,0.000591,0.015139,0.005422
2022-01-08,0.000116,0.002868,0.001995,0.013251,0.009516,0.021548,0.016848,0.000459,0.126060,0.006009,...,0.011592,0.013588,0.000998,2.483527e-09,0.000718,0.009696,0.000133,0.001231,0.013499,0.004148


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,0.000132,0.000825,0.001152,0.000900,0.004721,0.000033,0.000367,0.000459,0.006912,0.001398,...,0.001462,0.000549,0.000005,0.0,0.000289,0.005899,0.001071,0.000378,0.002528,0.003508
2022-01-07,0.000134,0.001187,0.001130,0.000450,0.005140,0.000017,0.000362,0.000480,0.006903,0.000952,...,0.001498,0.000436,0.001012,0.0,0.000519,0.006412,0.000536,0.000189,0.002322,0.004495
2022-01-08,0.000135,0.000593,0.000960,0.000225,0.005334,0.000008,0.000339,0.000365,0.009656,0.000982,...,0.001794,0.000299,0.001850,0.0,0.000540,0.006748,0.000268,0.000095,0.002216,0.003416


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-06,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078
2022-01-07,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078
2022-01-08,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-06,0.011670,0.003658,2.728484e-12,0.006615,0.011797,0.012713,0.009278,0.018201,0.011887,3.557910e-200,...,0.002691,0.010828,0.011856,0.024590,0.048352,0.013290,0.014693,0.012044,0.011123,0.007088
2022-01-07,0.012699,0.013194,1.364242e-12,0.008512,0.012969,0.015551,0.009730,0.018039,0.013374,1.778955e-200,...,0.001345,0.010781,0.012967,0.025222,0.037064,0.014257,0.016696,0.011969,0.012209,0.006752
2022-01-08,0.010699,0.006597,6.821210e-13,0.009966,0.013239,0.010637,0.004865,0.009019,0.016080,8.894776e-201,...,0.000673,0.010410,0.006484,0.030354,0.035133,0.007128,0.008348,0.011933,0.006104,0.003376


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-01-06      0.011824  0.011090  0.012753  0.010034  0.006530  0.015116   
2022-01-07      0.012400  0.011948  0.012350  0.014002  0.007709  0.014008   
2022-01-08      0.009041  0.010067  0.009790  0.011363  0.006378  0.013925   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-01-06      0.015439  0.008587  0.015741  0.007578  ...  0.002918   
2022-01-07      0.015276  0.008927  0.014383  0.009343  ...  0.003460   
2022-01-08      0.010590  0.007952  0.011676  0.007347  ...  0.001730   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-01-06      0.002287  0.004733  0.003137   0.004685  0.031453  0.005525   
2022-01-07      0.002084  0.004647  0.002861   0.004409  0.029550  0.005000   
2022-01-08      0.001042  0.002323  0.001431   0.002204  0.014775  0.002500   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-01-06           -1.0  0.003638  0.001156  
2022-01-07           -1.0  0.003391  0.001371  
2022-01-08           -1.0  0.001695  0.000686  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-06,0.001633,0.000068,NaN,0.001857,0.002079,0.000805,0.003476,0.001406,0.000014,NaN,...,0.000473,0.001472,0.005096,0.002283,0.000044,0.000699,0.004002,0.002761,0.004929,0.003768
2022-01-07,0.002086,0.000545,NaN,0.001539,0.002117,0.001926,0.002844,0.005875,0.009631,NaN,...,0.000236,0.001424,0.005004,0.001142,0.000022,0.000637,0.003505,0.004070,0.003607,0.001884
2022-01-08,0.002219,0.000272,NaN,0.002557,0.001919,0.001060,0.001422,0.002937,0.012326,NaN,...,0.000118,0.001341,0.002502,0.001608,0.000011,0.000319,0.001753,0.002035,0.001804,0.000942


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin   Barbour          Bibb    Blount   
2022-01-06      0.000781  0.003578  0.012346  3.099305e-13  0.000010   
2022-01-07      0.003496  0.004297  0.006173  1.549652e-13  0.002530   
2022-01-08      0.001748  0.003813  0.003086  7.748262e-14  0.008803   

Province_State                                                                \
Admin2               Bullock        Butler   Calhoun      Chambers  Cherokee   
2022-01-06      8.278424e-11  7.384089e-11  0.000004  2.238173e-07  0.001924   
2022-01-07      4.139212e-11  3.692044e-11  0.000942  1.119086e-07  0.000962   
2022-01-08      2.069606e-11  1.846022e-11  0.002347  5.595432e-08  0.000481   

Province_State  ...   Wyoming                                           \
Admin2          ...      Park    Platte  Sheridan  Sublette Sweetwater   
2022-01-06      ...  0.004274  0.007143  0.004098  0.005435   0.003505   
2022-01-07      ...  0.002137  0.003572  0.002049  0.002717   0.001752   
2022-01-08      ...  0.001068  0.001786  0.001025  0.001359   0.000876   

Province_State                                                         
Admin2                 Teton     Uinta Unassigned  Washakie    Weston  
2022-01-06      6.362931e-26  0.012097   0.752941  0.006757  0.017857  
2022-01-07      3.181466e-26  0.006048   0.752941  0.003378  0.008929  
2022-01-08      1.590733e-26  0.003024   0.752941  0.001689  0.004464  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 